In [1]:
# Import Dependencies

from time import sleep
# from features import get_features
from mido import MidiFile
import pandas as pd

from datetime import date
from time import time
from os import path, mkdir, listdir, environ
import subprocess


In [2]:
# Get Args, define evaluation groups

iterations = 1        # How many times each experiment should be repeated for the same parameters?
max_input_bars = 1    # What is the maximum lookback size for each model?
max_output_bars = 1   # What is the maximum output size for each model?
sample_length = 32    # How long of a final composition should be generated?

client  = None        # Client Process
server  = None        # Server Process
expname = ''          # Experiment Name, for naming files

ip = "127.0.0.1"
port = 5005
port_in = 57120

evaluation_sets = [
  ('bach-chorales', [
    ('polyphony_rnn', 'polyphony_rnn'),
    ('pianoroll_rnn_nade', 'rnn-nade_attn'),
    ('rl_duet', 'rl_duet'),
  ]),
  ('piano-e-comp', [
    ('performance_rnn', 'performance_with_dynamics'),
    # ('music_transformer', 'performance_with_dynamics')
  ]),
  ('None', [
    # ('melody_rnn', 'attention_rnn')
  ]),
  ('None', [
    # ('remi', 'remi')
  ])
]

In [3]:
# Folder Definitions
primerdir = ''
outputdir = path.join(path.pardir,'output')
datasetdir = path.join(path.pardir,'dataset') 
basefoldername = str(date.today())
i = 0
while True:
    foldername = f'{basefoldername}-{i}'
    full_foldername = path.join(outputdir, foldername)

    if not path.exists(full_foldername):
      mkdir(full_foldername)
      break

    if not any(listdir(full_foldername)): break
    i = i + 1


# File Management
def get_filename(expname,index,primer=None):
  return path.normpath(
    f"{foldername}/{expname}-{index}"
    if primer is None else
    f"{foldername}/{expname}-{primer}-{index}")

def get_primer_filename(name):
  return path.join(primerdir,f'{name}.mid')

def get_midi_filename(expname,index,primer=None):
  return path.join(outputdir, f'{get_filename(expname,index,primer)}.mid')

def get_pickle_filename(expname,index,primer=None):
  return path.join(outputdir, f'{get_filename(expname,index,primer)}.pkl')

def save_df(df, filename):
    # log(f'Saving dataframe to {filename}')
    df.to_pickle(filename)

def log(message):
    print(f'[batch:{expname}] {message}')

In [4]:
# Define Experiments

# EXPERIMENT 1: Output precision based on primer length
def analysis_input_length(primer):
  bars_output = 4  # vary output length
  client.set('voices', 1, 2)

  for j in range(1,max_input_bars):
      bars_input = j # Vary input length
      client.reset()
      client.load_bars(get_primer_filename(primer), bars_input)
      for b in range(bars_output): client.generate(1, 'measures')
      filename = get_filename(expname,i,f'{primer}-{j}-bars')
      log(f'save output to: {filename}')
      client.save(filename)  
    

# EXPERIMENT 2: Output precision based on requested output length
def analysis_output_length(primer):
  bars_input = 4 # Fixed input length
  client.set('buffer_length', 4)
  for j in range(1,max_output_bars):
    bars_output = j # Vary output length
    client.reset()
    client.load_bars(get_primer_filename(primer), bars_input)
    for b in range(bars_output): client.generate(1, 'measures')
    filename = get_filename(expname,i,f'{primer}-{j}-bars')
    log(f'save output to: {filename}')
    client.save(filename)

experiments = [
  ('input_len', analysis_input_length),
  ('output_len', analysis_output_length)
  ]

def experiment(name, fn):
  expname = name
  
  if df is None:
    cols = [ "model", "checkpoint", "primer", "out_file", "time", "in_len", "out_len" ]
    df = pd.DataFrame([],columns=cols)

  result = [fn(primer) for primer in primers]
  for primer in primers:
    for i in range(args.iterations):
      fn(primer)
  # TODO: save pickle

In [5]:
# Start Client, define model management functions
from pythonosc import udp_client, osc_server
from pythonosc.dispatcher import Dispatcher

class BatchClient(udp_client.SimpleUDPClient):
  def __init__(self, logger, ip, port_out, port_in):
    udp_client.SimpleUDPClient.__init__(self, ip, port_out, port_in)
    dispatcher = Dispatcher()
    dispatcher.map('/ok', lambda _: self.server.shutdown())
    self.server = osc_server.ThreadingOSCUDPServer((ip, port_in), dispatcher)
    self.log = logger

  def start(self):
    self.send_message('/start', [])

  def stop(self):
    self.send_message('/stop', [])

  def set(self,*args):
    self.send_message('/set', args)

  def debug(self,key):
    self.send_message('/debug', [key])

  def save(self,filename):
    self.send_message('/save', [filename])
    self.wait()

  def play(self,note):
    self.send_message('/play', [note + 40])

  def run(self,filename):
    self.set('output_filename', filename)
    self.start() # TODO: on host: unset 'batch_complete'
    self.wait()

  def reset(self):
    self.send_message('/reset', [])
  
  def pause(self):
    self.send_message('/pause', [])

  def end(self):
    self.send_message('/end', [])

  def generate(self, length, unit):
    self.send_message('/generate', [length, unit])
    self.wait()

  def wait(self):
    self.log("waiting...")
    self.server.serve_forever()
    self.log("ok!")
  
  def load_bars(self,filename,barcount):
    self.send_message('/load_bars', [filename, barcount])
    self.wait()

client = BatchClient(log, ip,  port,  port_in)
def start_model(modelname, checkpointname):
  modelname = modelname
  log(f'Starting model: {modelname} ({checkpointname})')
  server = subprocess.Popen([ './start.sh', model, checkpoint ], stdout=subprocess.PIPE, cwd=path.pardir)
  client.wait()
  client.set('debug_output', False)
  client.set('batch_mode', True)
  client.set('trigger_generate', 1)
  client.set('batch_unit', 'measures')
  client.set('debug_output', False)
  pass

def stop_model():
  if client: client.end()
  if server: server.wait()


In [6]:
# EXPERIMENT BLOCK
try:
    log(f'Starting experiment suite: {outputdir}/{foldername}')

    for dataset, models in evaluation_sets:
        datapath = path.join(datasetdir, dataset)
        if not path.exists(datapath):
            print(f'Directory not found: {datapath}, skipping')
            continue

        primers = listdir(datapath)[:1] # FIXME: Quick Test
        # primers = listdir(datapath)

        for (model, checkpoint) in models:
            start_model(model, checkpoint)
            for i in range(iterations):
                for name, exp in experiments:
                    experiment(name, exp)
            stop_model()

except KeyboardInterrupt:
  print("Terminating...")
  client.pause()
finally:
  stop_model()

print("Done!")

[batch:] Starting experiment suite: ../output/2021-05-17-0
Directory not found: ../dataset/bach-chorales, skipping
[batch:] Starting model: performance_rnn (performance_with_dynamics)


FileNotFoundError: [Errno 2] No such file or directory: './start.sh': './start.sh'